Notebook to determine the upper and lower bounds in the dynamic range for GHI estimation.

Four strategies are applied:
1) 90 days moving 2) 60 days moving 3) 30 days moving 4) monthly fixed

In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pvlib
import datetime

In [2]:
df = pd.read_csv('dra_c02.csv') # read the data, at dra, band 2, for other bands and locations, the procedure is the same.

df['time'] = pd.to_datetime(df['round_time_5min']) # select the 5-min nearest rounded time as the index. 
df = df.set_index(df['time'])
df = df.drop(columns=['time','Channel','time.1','round_time_1min','round_time_5min'])

df = df[df['1x1']>0]
df.head()

,1x1,3x3,5x5,kappa0
time,,,,
2019-01-01 00:05:00,10.635601,9.983610,10.178123,0.001862
2019-01-01 00:10:00,8.098124,8.750114,8.976481,0.001862
2019-01-01 00:15:00,8.256716,8.309580,8.281115,0.001862
2019-01-01 00:20:00,6.670792,6.847006,7.103871,0.001862
2019-01-01 00:25:00,5.877831,5.507782,5.542347,0.001862


In [3]:
# get the airmss and zenith using pvlib
times = pd.date_range(start = df.index[0],
                      end = df.index[-1],
                      freq = '5min')
latitude, longitude, name, altitude, timezone = 36.62373, -116.01947 ,'DRA', 1007, 0 # the time is in UTC, so the timezone is 0.
solpos = pvlib.solarposition.get_solarposition(times, latitude, longitude) # get solar position.
airmass = pvlib.atmosphere.get_relative_airmass(solpos['apparent_zenith']) # get the airmass.
pressure = pvlib.atmosphere.alt2pres(altitude) # pressure changes with altitude.
am_abs = pvlib.atmosphere.get_absolute_airmass(airmass, pressure) # absolute airmass.

In [4]:
df = pd.concat([df,am_abs.rename('am'),solpos['apparent_zenith'].rename('app_zen')],axis=1,join='inner')
df.head()

,1x1,3x3,5x5,kappa0,am,app_zen
2019-01-01 00:05:00,10.635601,9.983610,10.178123,0.001862,9.052654,84.945675
2019-01-01 00:10:00,8.098124,8.750114,8.976481,0.001862,10.425038,85.761186
2019-01-01 00:15:00,8.256716,8.309580,8.281115,0.001862,12.239576,86.576830
2019-01-01 00:20:00,6.670792,6.847006,7.103871,0.001862,14.708669,87.388895
2019-01-01 00:25:00,5.877831,5.507782,5.542347,0.001862,18.168317,88.191578


In [5]:
# add the sun earth distance for each day
DOY = [int(index.strftime('%j')) for index in df.index]
df_e = pd.read_csv('sun_earth_distance.csv')
df_e = df_e.set_index(df_e['DOY'])
dse = [df_e.loc[d,'d'] for d in DOY]
df['d_se'] = dse
df.head()

,1x1,3x3,5x5,kappa0,am,app_zen,d_se
2019-01-01 00:05:00,10.635601,9.983610,10.178123,0.001862,9.052654,84.945675,0.98331
2019-01-01 00:10:00,8.098124,8.750114,8.976481,0.001862,10.425038,85.761186,0.98331
2019-01-01 00:15:00,8.256716,8.309580,8.281115,0.001862,12.239576,86.576830,0.98331
2019-01-01 00:20:00,6.670792,6.847006,7.103871,0.001862,14.708669,87.388895,0.98331
2019-01-01 00:25:00,5.877831,5.507782,5.542347,0.001862,18.168317,88.191578,0.98331


In [6]:
# first normalization, 1x1 is the measured irradiance at the target location, while 3x3 and 5x5 are the averages.
df['norrad_1'] = df['1x1']*df['am']*df['d_se']
df['norrad_3'] = df['3x3']*df['am']*df['d_se']
df['norrad_5'] = df['5x5']*df['am']*df['d_se']
df.head()

,1x1,3x3,5x5,kappa0,am,app_zen,d_se,norrad_1,norrad_3,norrad_5
2019-01-01 00:05:00,10.635601,9.983610,10.178123,0.001862,9.052654,84.945675,0.98331,94.673493,88.869756,90.601226
2019-01-01 00:10:00,8.098124,8.750114,8.976481,0.001862,10.425038,85.761186,0.98331,83.014229,89.697808,92.018306
2019-01-01 00:15:00,8.256716,8.309580,8.281115,0.001862,12.239576,86.576830,0.98331,99.372034,100.008268,99.665679
2019-01-01 00:20:00,6.670792,6.847006,7.103871,0.001862,14.708669,87.388895,0.98331,96.480873,99.029481,102.744575
2019-01-01 00:25:00,5.877831,5.507782,5.542347,0.001862,18.168317,88.191578,0.98331,105.007966,98.397010,99.014510


In [7]:
# second normalization to account for the high airmass effect.
time_index = []
nrad1 = []
nrad3 = []
nrad5 = []
for index in df.index:
    elevation = 90-df.loc[index,'app_zen']
    if elevation >= 1.5:
        if elevation <=65:
            h = elevation
        else:
            h = 65
        nrad_1 = df.loc[index,'norrad_1']/(2.283*h**(-0.26)*np.exp(0.004*h))
        nrad_3 = df.loc[index,'norrad_3']/(2.283*h**(-0.26)*np.exp(0.004*h))
        nrad_5 = df.loc[index,'norrad_5']/(2.283*h**(-0.26)*np.exp(0.004*h))
        time_index.append(index)
        nrad1.append(nrad_1)
        nrad3.append(nrad_3)
        nrad5.append(nrad_5)

df_nr = pd.DataFrame()
df_nr['time'] = time_index
df_nr['nrad_1'] = nrad1
df_nr['nrad_3'] = nrad3
df_nr['nrad_5'] = nrad5
df_nr = df_nr.set_index(df_nr['time'])
df_nr.head()

,time,nrad_1,nrad_3,nrad_5
time,,,,
2019-01-01 00:05:00,2019-01-01 00:05:00,61.929082,58.132665,59.265277
2019-01-01 00:10:00,2019-01-01 00:10:00,52.043512,56.233600,57.688373
2019-01-01 00:15:00,2019-01-01 00:15:00,59.123894,59.502438,59.298605
2019-01-01 00:20:00,2019-01-01 00:20:00,53.675201,55.093069,57.159887
2019-01-01 00:25:00,2019-01-01 00:25:00,53.268775,49.915148,50.228395


In [8]:
# since the values of 1x1, 3x3, and 5x5 are have limited difference, 1x1 is used for GHI estimation
df = pd.concat([df,df_nr],axis=1,join='inner')
df = df[['1x1','app_zen','am','norrad_1','nrad_1']]
df.head()

,1x1,app_zen,am,norrad_1,nrad_1
2019-01-01 00:05:00,10.635601,84.945675,9.052654,94.673493,61.929082
2019-01-01 00:10:00,8.098124,85.761186,10.425038,83.014229,52.043512
2019-01-01 00:15:00,8.256716,86.576830,12.239576,99.372034,59.123894
2019-01-01 00:20:00,6.670792,87.388895,14.708669,96.480873,53.675201
2019-01-01 00:25:00,5.877831,88.191578,18.168317,105.007966,53.268775


In [9]:
# remove the data when the solar zenith is greater than 80.
df0 = df[df['app_zen']<=80]
df0.head()

,1x1,app_zen,am,norrad_1,nrad_1
2019-01-01 16:05:00,37.279121,79.316620,4.654689,170.626610,132.570280
2019-01-01 16:10:00,40.133781,78.551163,4.362383,172.156839,135.771201
2019-01-01 16:15:00,43.781410,77.794940,4.107490,176.830325,141.367264
2019-01-01 16:20:00,46.160294,77.048373,3.883555,176.274074,142.688040
2019-01-01 16:25:00,48.856369,76.311869,3.685506,177.055226,144.968486


In [11]:
# fixed month, the upper bound equals to the mean of the 10 highest values, 
#the lower bound is the mean of the second to the fifth values. 

df_ci = pd.DataFrame()
month = [1,2,3,4,5]

for mon in month:
    
    df_m = pd.DataFrame()
    index = [ind for ind in df0.index if ind.month == mon and ind.year == 2019]
    df_ = df0.loc[index]
    
    high = df_['nrad_1'].sort_values(ascending=False)[:10].mean()
    time_ = []
    ci_ = []
    
    for ind in df_.index:
        hour = ind.hour
        minute = ind.minute
        index_t = [time for time in df_.index if time.hour == hour and time.minute == minute]
        df_t = df.loc[index_t]
        
        try:
            low = df_t['nrad_1'].sort_values(ascending=True)[2:5].mean()
            ci = (df_.loc[ind,'nrad_1']-low)/(high-low)
            time_.append(ind)
            ci_.append(ci)
        except:
            print(ind)
        
    df_m['time'] = time_
    df_m['ci'] = ci_
    df_m = df_m.set_index('time')
        
    df_ci = pd.concat([df_ci,df_m],axis=0)
    
df_ci.head()  

,ci
time,
2019-01-01 16:05:00,0.013135
2019-01-01 16:10:00,0.067965
2019-01-01 16:15:00,0.013846
2019-01-01 16:20:00,-0.009069
2019-01-01 16:25:00,0.013379


In [12]:
# 90 days moving, the upper bound equals to the mean of the 20 highest values, 
# the lower bound is the mean of the 40 lowest values.

df_ci_90 = pd.DataFrame()
time_ci = []
ci_1 = []

for index in df0.index:
    dt = index
    dt_ = dt - datetime.timedelta(days=90)

    if (dt_ in df0.index):
        times = pd.date_range(start=dt_,end=dt,freq='5min')
        times = [time for time in times if time in df0.index]
        df_ = df0.loc[times]
        
        high_1 = df_['nrad_1'].sort_values(ascending=False)[:20].mean()
        
        times_ = pd.date_range(start = dt_, end = dt, freq = '1d')
        times_ = [time for time in times_ if time in df0.index]
            
        df_low = df0.loc[times_]
            
        low_1 = df_low['nrad_1'].sort_values(ascending=True)[:40].mean()
        
        ci1 = (df0.loc[dt,'nrad_1'] - low_1)/(high_1 - low_1)
        
        time_ci.append(index)
        ci_1.append(ci1)
        
df_ci_90['time'] = pd.to_datetime(time_ci)
df_ci_90 = df_ci_90.set_index('time')
df_ci_90['ci_1'] = ci_1
df_ci_90.head()   

,ci_1
time,
2019-04-01 16:05:00,0.048405
2019-04-01 16:10:00,0.115035
2019-04-01 16:15:00,0.131605
2019-04-01 16:20:00,0.157543
2019-04-01 16:25:00,0.117797


In [13]:
# 60 days moving, the upper bound equals to the mean of the 20 highest values, 
# the lower bound is the mean of the 40 lowest values.

df_ci_60 = pd.DataFrame()
time_ci = []
ci_1 = []

for index in df0.index:
    dt = index
    dt_ = dt - datetime.timedelta(days=60)
    
    if (dt_ in df0.index):
        times = pd.date_range(start=dt_,end=dt,freq='5min')
        times = [time for time in times if time in df0.index]
        df_ = df0.loc[times]
        
        high_1 = df_['nrad_1'].sort_values(ascending=False)[:20].mean()
        
        times_ = pd.date_range(start = dt_, end = dt, freq = '1d')
        times_ = [time for time in times_ if time in df0.index]
            
        df_low = df0.loc[times_]
            
        low_1 = df_low['nrad_1'].sort_values(ascending=True)[:40].mean()
        
        ci1 = (df0.loc[dt,'nrad_1'] - low_1)/(high_1 - low_1)
        
        time_ci.append(index)
        ci_1.append(ci1)
        
df_ci_60['time'] = pd.to_datetime(time_ci)
df_ci_60['ci_1'] = ci_1
df_ci_60 = df_ci_60.set_index('time')
df_ci_60.head()

,ci_1
time,
2019-03-02 16:05:00,0.772927
2019-03-02 16:10:00,0.638585
2019-03-02 16:15:00,0.634215
2019-03-02 16:20:00,0.527678
2019-03-02 16:25:00,0.567743


In [14]:
# 30 days moving, the upper bound equals to the mean of the 10 highest values, 
# the lower bound is the mean of the second to the fifth values.

df_ci_30 = pd.DataFrame()
time_ci = []
ci_1 = []

for index in df0.index:
    dt = index
    dt_ = dt - datetime.timedelta(days=30)

    if (dt_ in df0.index):
        times = pd.date_range(start=dt_,end=dt,freq='5min')
        times = [time for time in times if time in df0.index]
        df_ = df0.loc[times]
        
        high_1 = df_['nrad_1'].sort_values(ascending=False)[:10].mean()
        
        times_ = pd.date_range(start = dt_, end = dt, freq = '1d')
        times_ = [time for time in times_ if time in df0.index]
            
        df_low = df0.loc[times_]
            
        low_1 = df_low['nrad_1'].sort_values(ascending=True)[2:5].mean()
        
        ci1 = (df0.loc[dt,'nrad_1'] - low_1)/(high_1 - low_1)
        
        time_ci.append(index)
        ci_1.append(ci1)
        
df_ci_30['time'] = pd.to_datetime(time_ci)
df_ci_30['ci_1'] = ci_1
df_ci_30 = df_ci_30.set_index('time')
df_ci_30.head()

,ci_1
time,
2019-01-31 16:05:00,0.222082
2019-01-31 16:10:00,0.327348
2019-01-31 16:15:00,0.288201
2019-01-31 16:20:00,0.112097
2019-01-31 16:25:00,0.307581
